# Exploration for Reported Assessment Results and Suppressed Assessment Information

## Imports and Such

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# Import assessment data minus fully suppressed scores
assessments = pd.read_pickle('../data/school_based/assessments_clean.pkl')

# Import Suppressed outlier data
suppressed = pd.read_pickle ('../data/school_based/full_suppression.pkl')

# Import Tennessee School District Geometry
tn_leas = gpd.read_file('../data/tn_leas.shp')

## Listy McListface - A Place to look at the lists in my dataframe.

In [3]:
# Assessments Info
assessments.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 335452 entries, 0 to 381480
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   year                335452 non-null  int64   
 1   system_name         335452 non-null  object  
 2   school_name         335452 non-null  object  
 3   test                335452 non-null  object  
 4   subject             335452 non-null  object  
 5   subject_area        335452 non-null  object  
 6   student_group       335452 non-null  object  
 7   valid_tests         335452 non-null  float64 
 8   pct_met_exceeded    335452 non-null  float64 
 9   system              335452 non-null  object  
 10  school              335452 non-null  object  
 11  school_lvl          335452 non-null  object  
 12  tot_enrolled        335452 non-null  object  
 13  fte_teachers        334603 non-null  float64 
 14  stu_tchr_ratio      334603 non-null  float64 
 15  school_type   

In [4]:
# Student Groups
student_group_list = np.unique(assessments['student_group'].values).tolist()
student_group_list

['All Students',
 'American Indian or Alaska Native',
 'Asian',
 'Black or African American',
 'Black/Hispanic/Native American',
 'Economically Disadvantaged',
 'English Learner Transitional 1-4',
 'English Learners',
 'English Learners with Transitional 1-4',
 'Female',
 'Gifted',
 'Hispanic',
 'Male',
 'Native Hawaiian or Other Pacific Islander',
 'Non-Black/Hispanic/Native American',
 'Non-Economically Disadvantaged',
 'Non-English Learners/Transitional 1-4',
 'Non-Students with Disabilities',
 'Students with Disabilities',
 'Super Subgroup',
 'White']

In [5]:
# School Types
school_type_list = np.unique(assessments['school_type'].values).tolist()
school_type_list

['1-Regular school',
 '2-Special education school',
 '4-Alternative Education School']

In [6]:
# Subject Areas
subject_area_list = np.unique(assessments['subject_area'].values).tolist()
subject_area_list

['ELA', 'Math', 'Science', 'Social Studies']

## Unsuppressed: Broad Overview of Results

### Overall Reported Proficiencies by School-Level, Subject Area, and Year.

In [7]:
# Subject Area Pivot Table
subject_area_pivot = pd.pivot_table(assessments,
                       values = 'pct_met_exceeded',
                       index = ['school_lvl', 'subject_area'],
                       columns = 'year',
                       aggfunc = np.mean)

# Get the current list of years
years = list(subject_area_pivot.columns)

# Include 2020 as nan values to show a gap when testing did not occur
years.insert(years.index(2021), 2020)

# Reindex and set the columns to years
subject_area_pivot = subject_area_pivot.reindex(columns=years)

# Reset index to convert multi-index into columns
subject_area_pivot.reset_index(inplace=True)

subject_area_pivot

year,school_lvl,subject_area,2018,2019,2020,2021,2022
0,Elementary,ELA,0.313354,0.317439,NaN,0.288133,0.332412
1,Elementary,Math,0.351674,0.402860,NaN,0.308109,0.338107
2,Elementary,Science,0.519245,NaN,NaN,0.353617,0.367404
3,Elementary,Social Studies,0.338689,0.345175,NaN,0.328732,0.364729
4,High,ELA,0.269997,0.331378,NaN,0.294335,0.348687
5,High,Math,0.231409,0.260610,NaN,0.201373,0.229202
6,High,Science,0.400077,NaN,NaN,0.349336,0.355829
7,High,Social Studies,0.270426,0.286531,NaN,0.305459,0.343403
8,Middle,ELA,0.272938,0.289341,NaN,0.244374,0.294892
9,Middle,Math,0.362565,0.390276,NaN,0.341143,0.357363


### 🏋️ Weighting Metrics Based on Valid Tests

#### 🏫 List of columns in assessments (school level) for use in district analysis.

In [8]:
# Let's figure how how I can pivot this    
districts = assessments[['locale',
                         'year',
                         'system_name', 
                         'school_lvl', 
                         'subject_area', 
                         'student_group', 
                         'pct_met_exceeded_w', 
                         'school_type',
                         'magnet',
                         'charter',
                         'title_1',
                         'fte_teachers_w',
                         'stu_tchr_ratio_w',
                         'valid_tests']]

#### 📇 Indices for Weight Pivots

In [9]:
# Indices for pivots
indices = ['system_name',  # School District 
           'school_lvl', # Level of school (Elem, Middle, High)
           'school_type', # 
           'magnet', 
           'charter', 
           'title_1', 
           'locale', 
           'subject_area', 
           'student_group']

# # Calculate change in valid test scores from the year before schools closed down to the year after
# # Negative values indicate decrease from pre-pandemic. Positive values indicate an increase
# weight['1y_post'] = weight[2021] - weight[2019]
# # Calculate the change in valid test scores 2 years post-pandemic
# # Negative values indicate decrease in the first two years post-pandemic
# weight['2y_post'] = weight[2022] - weight[2021]
# # Calculate the difference between the current valid test scores and pre-pandemic
# # Negative values indicate scores are below pre-pandemic levels
# weight['pre_current'] = weight[2022] - weight[2019]
# # Look at the pivot
# weight

#### 🏋️➕ Sum of Valid_Test-Weighted Scores for pct_met_exceeded, fte_teachers, and student_tchr_ratio 

In [10]:
# Sum of valid test scores (The 🏋️)
weight = pd.pivot_table(
    districts,
    values='valid_tests',  # Column to calculate the sum of valid test scores
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'pct_met_exceeded'
sum_weighted_proficiency = pd.pivot_table(
    districts,
    values='pct_met_exceeded_w', # Weighted sum of students who met or exceeded expectations
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'fte_teachers'
sum_weighted_fte = pd.pivot_table(
    districts,
    values='fte_teachers_w', # Weighted sum of full-time equivalent teachers
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Sum of weighted scores pivot for 'stu_tchr_ratio'
sum_weighted_str = pd.pivot_table(
    districts,
    values='stu_tchr_ratio_w', # Weighted sum of student/teacher ratios
    index=indices,
    columns='year',
    aggfunc=np.sum
)

# Create a multi-level column index
column_index = pd.MultiIndex.from_product([['pct_met_exceeded', 'fte_teachers', 'stu_tchr_ratio'], sum_weighted_proficiency.columns])

# Concatenate the pivot tables horizontally
weighted_sums_pivot = pd.concat([sum_weighted_proficiency, sum_weighted_fte, sum_weighted_str], axis=1)
weighted_sums_pivot.columns = column_index

weighted_sums_pivot

pct_met_exceeded  \
year                                                                                                                                                            2018   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                            
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                                    36.032   
                                                                                                              Black or African American                       34.028   
                                                                                                              Black/Hispanic/Native American                  34.030   
                                                                                                              Economically Disadvantaged                      32.996   
                                                                                                              Female                                             NaN   
...                                                                                                                                                              ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4           11.000   
                                                                                                              Non-Students with Disabilities                  10.002   
                                                                                                              Students with Disabilities                         NaN   
                                                                                                              Super Subgroup                                     NaN   
                                                                                                              White                                           11.002   

                                                                                                                                                             \
year                                                                                                                                                   2019   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                   
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                           33.021   
                                                                                                              Black or African American              31.004   
                                                                                                              Black/Hispanic/Native American         31.097   
                                                                                                              Economically Disadvantaged             27.936   
                                                                                                              Female                                 18.988   
...                                                                                                                                                     ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4     NaN   
                                                                                                              Non-Students with Disabilities            NaN   
                                              

#### 🏋️⚖️ Weighted Averages for pct_met_exceeded_w, fte_teachers_w, stu_tchr_ratio_w

In [11]:
# Divide sum_weighted_proficiency by weight
weighted_avg_proficiency = sum_weighted_proficiency / weight

# Divide sum_weighted_fte by weight
weighted_avg_fte = sum_weighted_fte / weight

# Divide sum_weighted_str by weight
weighted_avg_str = sum_weighted_str / weight

# Create a multi-level column index for the weighted average pivots
column_index = pd.MultiIndex.from_product([['pct_met_exceeded', 'fte_teachers', 'stu_tchr_ratio'], weighted_avg_proficiency.columns])

# Concatenate the weighted average pivots horizontally
weighted_avg_pivot = pd.concat([weighted_avg_proficiency, weighted_avg_fte, weighted_avg_str], axis=1)
weighted_avg_pivot.columns = column_index

weighted_avg_pivot

pct_met_exceeded  \
year                                                                                                                                                            2018   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                            
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                                  0.090990   
                                                                                                              Black or African American                     0.090260   
                                                                                                              Black/Hispanic/Native American                0.088390   
                                                                                                              Economically Disadvantaged                    0.094816   
                                                                                                              Female                                             NaN   
...                                                                                                                                                              ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4         0.314286   
                                                                                                              Non-Students with Disabilities                0.322645   
                                                                                                              Students with Disabilities                         NaN   
                                                                                                              Super Subgroup                                     NaN   
                                                                                                              White                                         0.343813   

                                                                                                                                                               \
year                                                                                                                                                     2019   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                     
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                           0.075048   
                                                                                                              Black or African American              0.072609   
                                                                                                              Black/Hispanic/Native American         0.071160   
                                                                                                              Economically Disadvantaged             0.074895   
                                                                                                              Female                                 0.104330   
...                                                                                                                                                       ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4       NaN   
                                                                                                              Non-Students with Disabilities              NaN   
                        

### 🤺 Unferths' Hrunting 

⚔️ `weighted_avg_pivot.loc[(slice(None), slice(None), slice(None),slice(None),slice(None),slice(None),slice(None), 'Science')]` ⚔️

🔖 `weighted_avg_pivot[('pct_met_exceeded', 'first lag')] = weighted_avg_pivot.loc[:, 'pct_met_exceeded'][2022] - weighted_avg_pivot.loc[:, 'pct_met_exceeded'][2018]` 🔖

In [12]:
# # Use this to test your calculations.
# weighted_avg_pivot[('pct_met_exceeded', 'first lag')] = weighted_avg_pivot.loc[:, 'pct_met_exceeded'][2021] - weighted_avg_pivot.loc[:, 'pct_met_exceeded'][2019] 
# weighted_avg_pivot.loc[:, 'pct_met_exceeded']

Variables created to greatly minimize repitition of literal slices

In [13]:
# Set the stop year for all metrics
lvpp_stop = 2021

# Set the start year for Math, ELA, and Social Studies related metrics
lvpp_start_mess = 2019

# Set the start year for Science-related metrics
lvpp_start_science = 2018

# Set the start and stop year for the "intra" lag
intra_start = 2021
intra_stop = 2022

# Science Slicer
science_slice = (slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), 'Science')

# Non-Science slicer
subjects_slice = (slice(None), slice(None), slice(None), slice(None), slice(None), slice(None), slice(None))

#### 📝 Assessment Lags

In [14]:
# Calculate separate lag scores for Science-related metrics.

# Last Valid Pre-Pandemic met_exceded measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('pct_met_exceeded', lvpp_start_science)]
)

# Calculate the lvpp scores for Math, ELA, and Social Studies related metrics
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('pct_met_exceeded', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('pct_met_exceeded', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('pct_met_exceeded', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'pct_met_exceeded'
weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('pct_met_exceeded', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores
weighted_avg_pivot[('pct_met_exceeded', 'pre-post')] = (
    weighted_avg_pivot[('pct_met_exceeded', 2022)] -
    weighted_avg_pivot[('pct_met_exceeded', 'lvpp')]
)

#### 🧑‍🏫 Full Time Equivalent Teachers Lag

In [15]:
# Last Valid Pre-Pandemic science-related full-time equivalent teachers measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('fte_teachers', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('fte_teachers', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('fte_teachers', lvpp_start_science)]
)

# Last Valid Pre-Pandemic full-time equivalent teacher measurement (lvpp)
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('fte_teachers', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('fte_teachers', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('fte_teachers', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'fte_teachers'
weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('fte_teachers', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores for full-time equivalent teachers
weighted_avg_pivot[('fte_teachers', 'pre-post')] = (
    weighted_avg_pivot[('fte_teachers', 2022)] -
    weighted_avg_pivot[('fte_teachers', 'lvpp')]
)

#### 🧑‍🎓/🧑‍🏫 Student Teacher Ratio Lag

In [16]:
# Last Valid Pre-Pandemic science-related student/teacher ratio measurement (lvpp)
weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', 'lvpp')] = (
    weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', lvpp_stop)] -
    weighted_avg_pivot.loc[science_slice, ('stu_tchr_ratio', lvpp_start_science)]
)

# Last Valid Pre-Pandemic non science-related student/teacher ratio measurement (lvpp)
weighted_avg_pivot.loc[
    subjects_slice + (['Math', 'ELA', 'Social Studies'],),
    ('stu_tchr_ratio', 'lvpp')
] = (
    weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('stu_tchr_ratio', lvpp_stop)
    ] - weighted_avg_pivot.loc[
        subjects_slice + (['Math', 'ELA', 'Social Studies'],),
        ('stu_tchr_ratio', lvpp_start_mess)
    ]
)

# Calculate the intra lag score for 'stu_tchr_ratio'
weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', 'intra')] = (
    weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', intra_stop)] -
    weighted_avg_pivot.loc[subjects_slice, ('stu_tchr_ratio', intra_start)]
)

# Calculate the 'pre-post' assessment lag scores for student/teacher ratios
weighted_avg_pivot[('stu_tchr_ratio', 'pre-post')] = (
    weighted_avg_pivot[('stu_tchr_ratio', 2022)] -
    weighted_avg_pivot[('stu_tchr_ratio', 'lvpp')]
)


In [17]:
weighted_avg_pivot

pct_met_exceeded  \
year                                                                                                                                                            2018   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                            
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                                  0.090990   
                                                                                                              Black or African American                     0.090260   
                                                                                                              Black/Hispanic/Native American                0.088390   
                                                                                                              Economically Disadvantaged                    0.094816   
                                                                                                              Female                                             NaN   
...                                                                                                                                                              ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4         0.314286   
                                                                                                              Non-Students with Disabilities                0.322645   
                                                                                                              Students with Disabilities                         NaN   
                                                                                                              Super Subgroup                                     NaN   
                                                                                                              White                                         0.343813   

                                                                                                                                                               \
year                                                                                                                                                     2019   
system_name                 school_lvl school_type      magnet charter title_1 locale          subject_area   student_group                                     
Achievement School District Elementary 1-Regular school 0      0       1       11-City: Large  ELA            All Students                           0.075048   
                                                                                                              Black or African American              0.072609   
                                                                                                              Black/Hispanic/Native American         0.071160   
                                                                                                              Economically Disadvantaged             0.074895   
                                                                                                              Female                                 0.104330   
...                                                                                                                                                       ...   
Wilson County               Other      1-Regular school 0      0       0       31-Town: Fringe Social Studies Non-English Learners/Transitional 1-4       NaN   
                                                                                                              Non-Students with Disabilities              NaN   
                        

In [18]:
# #Calculate difference between last valid prepandemic test and the first valid post-covid score.

# # Last valid test for Science
# science_lvpp = 2018
# science_stop = 2021

# # Last valid test for Math
# math_lvpp = 2019
# math_stop = 2021

# # Last valid test for ELA
# math_lvpp = 2019
# math_stop = 2021

# # Last valid test for social studies
# social_studies_lvpp = 2019
# social_studies_stop = 2021

In [19]:
# Unstack weighted assessments pivot table
weighted_assessments = weighted_avg_pivot.stack().reset_index()
weighted_assessments

,system_name,school_lvl,school_type,magnet,charter,title_1,locale,subject_area,student_group,year,fte_teachers,pct_met_exceeded,stu_tchr_ratio
0,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2018,16.068687,0.090990,19.127374
1,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2019,13.614091,0.075048,19.758750
2,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2021,13.437050,0.057583,15.263022
3,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2022,10.265169,0.089914,18.669213
4,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,lvpp,-0.177041,-0.017465,-4.495728
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227185,Wilson County,Other,1-Regular school,0,0,0,31-Town: Fringe,Social Studies,Super Subgroup,intra,-81.680000,0.108000,-13.920000
227186,Wilson County,Other,1-Regular school,0,0,0,31-Town: Fringe,Social Studies,White,2018,5.500000,0.343813,10.000000
227187,Wilson County,Other,1-Regular school,0,0,0,31-Town: Fringe,Social Studies,White,2021,98.610000,0.571000,27.030000
227188,Wilson County,Other,1-Regular school,0,0,0,31-Town: Fringe,Social Studies,White,2022,16.930000,0.600000,13.110000


###  GeoSpatial Merge

In [20]:
weighted_assessments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227190 entries, 0 to 227189
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   system_name       227190 non-null  object 
 1   school_lvl        227190 non-null  object 
 2   school_type       227190 non-null  object 
 3   magnet            227190 non-null  int64  
 4   charter           227190 non-null  int64  
 5   title_1           227190 non-null  object 
 6   locale            227190 non-null  object 
 7   subject_area      227190 non-null  object 
 8   student_group     227190 non-null  object 
 9   year              227190 non-null  object 
 10  fte_teachers      227190 non-null  float64
 11  pct_met_exceeded  227190 non-null  float64
 12  stu_tchr_ratio    227190 non-null  float64
dtypes: float64(3), int64(2), object(8)
memory usage: 22.5+ MB


### 📊 Visual EDA

In [21]:
# Get unique school levels
school_levels = weighted_assessments['school_lvl'].unique()

# Create the heatmap traces for each school level
heatmaps = []
for school_level in school_levels:
    data = weighted_assessments[weighted_assessments['school_lvl'] == school_level]
    heatmap = go.Heatmap(
        x=data['subject_area'],
        y=data['year'],
        z=data['pct_met_exceeded'],
        colorscale='viridis',
        colorbar=dict(title='pct_met_exceeded'),
        name=school_level
    )
    heatmaps.append(heatmap)

# Create the figure and add the heatmaps
fig = go.Figure(data=heatmaps)

# Customize the layout
fig.update_layout(
    title='Changes in pct_met_exceeded by School Level, Subject Area, and Year',
    xaxis=dict(title='Subject Area'),
    yaxis=dict(title='Year'),
    height=600,
    width=800
)

# Show the facetted heatmap
fig.show()


NameError: name 'go' is not defined

In [ ]:
weighted_assessments.head()

,system_name,school_lvl,school_type,magnet,charter,title_1,locale,subject_area,student_group,year,fte_teachers,pct_met_exceeded,stu_tchr_ratio
0,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2018,16.068687,0.090990,19.127374
1,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2019,13.614091,0.075048,19.758750
2,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2021,13.437050,0.057583,15.263022
3,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,All Students,2022,10.265169,0.089914,18.669213
4,Achievement School District,Elementary,1-Regular school,0,0,1,11-City: Large,ELA,Black or African American,2018,16.022016,0.090260,19.146605


In [ ]:
subject_area = [weighted_assessments['subject_area'].unique()]
subject_area

[array(['ELA', 'Math', 'Science', 'Social Studies'], dtype=object)]

In [ ]:
# Calculate average proficiency per school level, content area, and year
average_proficiency = weighted_assessments.groupby(['school_lvl', 'subject_area', 'year'])['pct_met_exceeded'].mean().reset_index()

# Define subject_area
subject_area = weighted_assessments['subject_area'].unique().tolist()

# Reorder scool levels
school_lvl_order = ['Elementary', 'Middle', 'High', 'Secondary', 'Other']

# Create bar plot
fig = px.bar(average_proficiency, x='year', y='pct_met_exceeded', color='subject_area',
             facet_row='subject_area', facet_col='school_lvl',
             category_orders={'subject_area': subject_area, 'school_lvl': school_lvl_order},
             labels={'pct_met_exceeded': 'Average Proficiency',
                     'school_lvl': 'School Level',
                     'subject_area': 'Subject Area'})

# Update layout
fig.update_layout(
    title='Average Weighted Proficiency by School Level, Content Area, and Year',
   autosize=True,
    width=1200,
    height=900,
)

# Update the graph size to 50% Larger
fig.update_layout(
    autosize=True,
    width=1200,
    height=900,
)

# Show the figure
fig.show()
